# DART Open API를 이용해 기업 공시 정보를 가져온다. 

DART Open API 사용 연습

해당 코드는 Python 3.5/3.7, 32bit/64bit 상관 없다. 

In [149]:
# -*- coding: utf-8 -*-

from urllib.request import Request
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

import re
import json
import xml.etree.ElementTree as elemTree
import sys

## DART Open API와 연결

DART API에는 4가지 정보가 있고, 각 정보는 더 세부적으로 나뉜다. 
1. 공시정보
2. 사업보고서 주요정보
3. 상장기업 재무정보
4. 지분공시 종합정보

In [4]:
with open('./DART_password.txt', 'r') as f:
    API_KEY = f.read()

In [5]:
crtfc_key = '?crtfc_key=' + API_KEY

1. 공시정보: 
    - 공시검색: 공시 유형별, 회사별, 날짜별 등 여러가지 조건으로 공시보고서 검색기능을 제공합니다.
    - 기업개황: DART에 등록되어있는 기업의 개황정보를 제공합니다.
    - 공시서류원본파일: 공시보고서 원본파일을 제공합니다.
    - 고유번호: DART에 등록되어있는 공시대상회사의 고유번호,회사명,대표자명,종목코드, 최근변경일자를 파일로 제공합니다.

기타 세부사항은 API doc에서 확인: https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001

In [134]:
## 공시정보 base URLs

DART_list_json = 'https://opendart.fss.or.kr/api/list.json' # 공시검색
DART_company_json = 'https://opendart.fss.or.kr/api/company.json' # 기업개황
DART_document_xml = 'https://opendart.fss.or.kr/api/document.xml' # 공시서류원본파일
DART_corpCode_xml = 'https://opendart.fss.or.kr/api/corpCode.xml' # 고유번호

In [137]:
def to_parameter(key, value):
    return '&' + str(key) + '=' + str(value)

In [138]:
def DART_annc_info(info_type, **kwargs):
    parameters = ''
    for k, v in kwargs.items():
        parameters += to_parameter(k,v)
    
    if info_type == 'list':
        return DART_list_json + crtfc_key + parameters
    elif info_type == 'company':
        return DART_company_json + crtfc_key + parameters
    elif info_type == 'document':
        return DART_document_xml + crtfc_key + parameters
    elif info_type == 'corpCode':
        return DART_corpCode_xml + crtfc_key + parameters
    else:
        print('Wrong info_type. Choose from:')
        print('''
        1. "list": 공시검색
        2. "company": 기업개활
        3. "document": 공시서류원본파일
        4. "corpCode": 고유번호
        ''')

In [158]:
def DART_get_response(request_url):
    req = urlopen(request_url)
    response = req.read().decode('utf8')
    
    try:
        result = ('json', json.loads(response))
    except JSONDecodeError:
        result = ('xml', elemTree.fromstring(response))
    except:
        print("An error occurred: ", sys.exc_info()[0])
        return 0
        
    return result

In [154]:
req_url = DART_annc_info('list', corp_code='00919966', bgn_de='20190801', end_de='20200117')
req_url

'https://opendart.fss.or.kr/api/list.json?crtfc_key=407c1fe7fc7a1a183002c6d5f981408662cd879e&corp_code=00919966&bgn_de=20190801&end_de=20200117'

In [161]:
DART_get_response(req_url)

('json',
 {'status': '000',
  'message': '정상',
  'page_no': 1,
  'page_count': 10,
  'total_count': 19,
  'total_page': 2,
  'list': [{'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '주식명의개서정지(주주명부폐쇄)',
    'rcept_no': '20191216900607',
    'flr_nm': '신라젠',
    'rcept_dt': '20191216',
    'rm': '코'},
   {'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '주식등의대량보유상황보고서(일반)',
    'rcept_no': '20191213000531',
    'flr_nm': '문은상',
    'rcept_dt': '20191213',
    'rm': ''},
   {'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '주식등의대량보유상황보고서(약식)',
    'rcept_no': '20191206000517',
    'flr_nm': 'BlackRockFundAdvisors',
    'rcept_dt': '20191206',
    'rm': ''},
   {'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '분기보고서 (2019.09)',

## 공시시간 크롤링

문의결과, DART API는 현재 공시시간 정보를 제공하지 않는다. (아직 시범운영기간임을 감안하긴 해야한다.)

In [10]:
date = '2019.08.02'
recent_annc_url = 'http://dart.fss.or.kr/dsac001/mainK.do?selectDate={}&sort=&series=&mdayCnt=0#'.format(date)

recent_annc_req = urlopen(recent_annc_url)
recent_annc_bs = bs(recent_req, 'html.parser')
recent_annc_bs

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>최근공시</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- jquery 1.2.6 -->
<script src="/js/prototype.js" type="text/javascript"></script>
<script src="/js/jquery/jquery-all.js" type="text/javascript"></script>
<!-- 2011.11.01 ext 2.3 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><link href="/js/ext-main/resources/css/ext-all.css" rel="stylesheet" type="text/css"/><!--<![endif]-->
<script src="/js/ext-main/adapter/ext/ext-base.js" type="text/javascript"></script>
<script src="/js/ext-main/ext-all.js" type="text/javascript"></script>
<!-- x-xeries js libraries  -->
<script src="/js/xjs.js" type

재밌는 사실. 이 소스코드엔 애초에 tbody가 없었다. 그래서 한참 헤맸다. 

Chrome selector에선 #listContents > div.table_list > table > tbody > tr:nth-child(1) 라고 나왔지만 이는 브라우저가 insert시킨 것이다. 이런 점을 고려하여 selector를 써야한다. 무조건 Chrome에서 copy selector 한다고 되지 않는다. 

관련: https://stackoverflow.com/questions/20522820/how-to-get-tbody-from-table-from-python-beautiful-soup

In [98]:
recent_annc_list_bs = recent_bs.select('div.table_list > table > tr')
len(recent_annc_list_bs)

65

In [100]:
a = recent_annc_list_bs[0]
a

<tr>
<td class="cen_txt">
						17:54
					</td>
<td>
<span class="nobr1" style="max-width:150px;">
<img alt="코스닥시장" src="/images/ico_kosdaq.gif" title="코스닥시장"/> 
							<a href="/dsae001/selectPopup.ax?selectKey=00347877" onclick="openCorpInfo('00347877'); return false;" title="이에스에이 기업개황 새창">
								이에스에이
							</a>
</span>
</td>
<td>
<a href="/dsaf001/main.do?rcpNo=20190802900517" id="r_20190802900517" onclick="openReportViewer('20190802900517'); return false;" title="주주총회소집결의 공시뷰어 새창"><span title="본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임">[기재정정]</span>주주총회소집결의
							
  							
						</a>
</td>
<td title="이에스에이"><div class="nobr" style="width:95px">이에스에이</div></td>
<td class="cen_txt">2019.08.02</td>
<td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 코스닥시장본부 소관임" hspace="1" src="/images/sub/remark05.gif" title="본 공시사항은 한국거래소 코스닥시장본부 소관임"/> <img alt="본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람" hspace="1" src="/images/sub/remark01.gif" title="본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람"/></td>
</

In [114]:
regex = re.compile(r'\d\d:\d\d')
timeis = a.find('td', attrs={'class':'cen_txt'}).text
timeis

'\r\n\t\t\t\t\t\t17:54\r\n\t\t\t\t\t'

In [117]:
regex.search(timeis).group()

'17:54'

In [120]:
reg = re.compile(r'\d{8}')

In [123]:
corp_code = a.find('span', {'class':'nobr1'}).a.attrs['onclick']
corp_code

"openCorpInfo('00347877'); return false;"

In [124]:
reg.search(corp_code).group()

'00347877'

In [127]:
reg = re.compile(r'openReportViewer')

annc_content = a.find('a', attrs={'onclick':reg})
annc_content

<a href="/dsaf001/main.do?rcpNo=20190802900517" id="r_20190802900517" onclick="openReportViewer('20190802900517'); return false;" title="주주총회소집결의 공시뷰어 새창"><span title="본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임">[기재정정]</span>주주총회소집결의
							
  							
						</a>

In [128]:
annc_content.text

'[기재정정]주주총회소집결의\r\n\t\t\t\t\t\t\t\r\n  \t\t\t\t\t\t\t\r\n\t\t\t\t\t\t'

In [132]:
reg = re.compile(r'\d+')
annc_id = reg.search(annc_content.attrs['id']).group()
annc_id

'20190802900517'